In [5]:
import os
import openai
import sys

sys.path.append("../..")

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())  # read local .env file

openai.api_key = os.environ["OPENAI_API_KEY"]

Getting Started

In [6]:
import instructor
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import Optional

# This enables response_model keyword
# from client.chat.completions.create
client = instructor.patch(OpenAI())


class Age(BaseModel):
    chain_of_thought: str = Field(
        ...,
        description="Think step by step to determine the correct number of years old the user is.",
    )
    years: int


class UserDetail(BaseModel):
    """This is information about a user."""

    name: str = Field(description="The first and last name of the user.")
    age: Age
    occupation: Optional[str] = Field(description="The name of the users job.")


user = client.chat.completions.create(
    model="gpt-4-1106-preview",
    response_model=UserDetail,
    messages=[
        {
            "role": "user",
            "content": "Today is December 1, 2026. The first person to walk in the room is named Steve Farmpt. He is the same age as the comedian Mel Brooks. He's from the Cornell family. He works down at the shoe factory on the assembly line. Because he's big, people typically address him as Mr. Gantenbien.",
        },
    ],
)

In [7]:
user

UserDetail(name='Steve Farmpt', age=Age(chain_of_thought='To know the current age of Steve Farmpt, we need to determine the age of Mel Brooks as of 2026. Mel Brooks was born on June 28, 1926. By December 1, 2026, he would have celebrated his 100th birthday. Therefore, Steve Farmpt is also 100 years old.', years=100), occupation='assembly line worker at a shoe factory')

Handle error messages

In [8]:
# class UserDetail(BaseModel):
#     age: int
#     name: str
#     role: Optional[str] = Field(default=None)


class MaybeUser(BaseModel):
    result: Optional[UserDetail] = Field(default=None)
    error: bool = Field(default=False)
    message: Optional[str]

    def __bool__(self):
        return self.result is not None


def create_user_from_sentence(sentence):
    return client.chat.completions.create(
        model="gpt-3.5-turbo",
        response_model=MaybeUser,
        messages=[
            {"role": "user", "content": sentence},
        ],
    )

In [9]:
# user = create_user_from_sentence("No one is anybody")
user = create_user_from_sentence(
    "John was a mechanic since he was 10 years old in 1983."
)


if user:
    print(f"this was successful: {user.result}")
else:
    print(f"this was unsuccessful: {user.error} {user.message}")

this was successful: name='John' age=Age(chain_of_thought='mechanic', years=10) occupation='mechanic'


In [10]:
EasyMaybeUser = instructor.Maybe(UserDetail)


def create_user_from_sentence_easy(sentence):
    return client.chat.completions.create(
        model="gpt-3.5-turbo",
        response_model=EasyMaybeUser,
        messages=[
            {"role": "user", "content": sentence},
        ],
    )

In [11]:
user = create_user_from_sentence_easy("No one is anybody")
# user = create_user_from_sentence_easy("John was a mechanic since he was 10 years old in 1983.")


if user:
    print(f"this was successful: {user.result}")
else:
    print(f"this was unsuccessful: {user.error} {user.message}")

this was unsuccessful: True No user was found
